In [1]:
from experta import *

In [2]:
class Age(Fact):
    pass

class Height(Fact):
    pass

class Weight(Fact):
    pass

class Gender(Fact):
    pass 

class Calories(Fact):
    pass

class Competing(Fact):
    pass

class Intensity(Fact):
    pass

class DietPlan(Fact):
    pass

class BMI (Fact):
    pass

class setOptions(Fact):
    pass

class Option(Fact):
    foodSet = Field(str, default="")
    option = Field(str, default="")
    calories = Field(str, default="")

class Set(Fact):
    setCategory = Field(str, default="")
    setFoods = Field(list, default=[])
    setCalories = Field(str, default="")
    
class BMIValue(Fact):
    pass

class Ask(Fact):
    pass

In [3]:
class SystemDiet(KnowledgeEngine):
    @DefFacts()
    def system_rules(self):
        yield Option(foodSet="casual",option="C",calories=">=2500")
        yield Option(foodSet="casual",option="B",calories=">=1500 & <2500")
        yield Option(foodSet="casual",option="A",calories="<1500") 
        
        yield Option(foodSet="light",option="F",calories=">=3500 & <4500")
        yield Option(foodSet="light",option="E",calories=">=2500 & <3500")
        yield Option(foodSet="light",option="D",calories="<2500")
        
        yield Option(foodSet="moderate",option="I",calories=">=3500")
        yield Option(foodSet="moderate",option="H",calories=">=2500 & <3500")
        yield Option(foodSet="moderate",option="G",calories="<2500")
        
        yield Option(foodSet="intensive",option="L",calories=">=4500")
        yield Option(foodSet="intensive",option="K",calories=">=3500 & <4500")
        yield Option(foodSet="intensive",option="J",calories="<3500")
        
        
        yield Set(setCategory="A",setFoods=["Fatty/Oily Rice","charcoal-grilled chicken","chicken rice"],
                  setCalories="<1500")
        yield Set(setCategory="B",setFoods=["Flattened bread","arab rice mandy","Mamak-style noodles,"],
                  setCalories=">=1500 & <2500")
        yield Set(setCategory="C",setFoods=["rice with salad + Flattened bread","rice on the shoulder","Chicken Chop"],
                  setCalories=">=2500")
        
        yield Set(setCategory="D",setFoods=["Fatty/Oily Rice","charcoal-grilled chicken","satay, fried wet"],
                  setCalories="<2500")
        yield Set(setCategory="E",setFoods=["Flattened bread","rice dish flavored with spices and rose water","Spaghetti"],
                  setCalories=">=2500 & <3500")
        yield Set(setCategory="F",setFoods=["Flat rice noodles. + Flattened bread","rice on the shoulder","Fried rice"],
                  setCalories=">=3500 & <4500")
        
        yield Set(setCategory="G",setFoods=["Fatty/Oily Rice + Flattened bread","charcoal-grilled chicken","Pizza"],
                  setCalories="<2500")
        yield Set(setCategory="H",setFoods=["rice with salad + Flattened bread","Chicken Fried Rice + chewy and slippery","Fried rice with egg"],
                  setCalories=">=2500 & <3500")
        yield Set(setCategory="I",setFoods=["Flat rice noodles. + Fatty/Oily Rice + Flattened bread","arab rice mandy + Vermicelli soup",
                  "Spaghetti + Chicken Chop + satay, fried wet"],setCalories=">=3500")
        
        yield Set(setCategory="J",setFoods=["Flat rice noodles. + Fatty/Oily Rice + Cereal",
                  "rice on the shoulder + Nasi Ayam Goreng + Bihun Sup","satay, fried wet"],setCalories="<3500")
        yield Set(setCategory="K",setFoods=["Flattened bread + rice with salad + Cereal",
                  "rice dish flavored with spices and rose water + charcoal-grilled chicken + chewy and slippery","Mamak-style noodles, + Fried rice with egg + Pizza"],
                  setCalories=">=3500 & <4500")
    
        yield Set(setCategory="L",setFoods=["Flattened bread + Flattened bread + Flat rice noodles. + Cereal",
                  "rice on the shoulder + Chicken Fried Rice + Bihun Sup","Nasi Goreng Pattaya + Mamak-style noodles, + Chicken Chop"],
                  setCalories=">=4500")

    @Rule()
    def start(self):
        print("The system has been started")

    @Rule(AS.f1<<Gender(MATCH.g),
      AS.f2<<Age(MATCH.a),
      AS.f3<<Height(MATCH.h),
      AS.f4<<Weight(MATCH.w))
    def gender(self,g,a,h,w):
        if(g=="male"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))+5)
            self.declare(Calories(value = caloriesFloat))
            print("You are a " + g)

        elif(g =="female"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))+161)
            self.declare(Calories(value = caloriesFloat))
            print("You are a " + g)
        else:
            print("Invalid gender")
        self.declare(Ask("competition-participation"))

    @Rule(AS.f1<<Ask("competition-participation"))
    def if_competing(self,f1):
        self.retract(f1)
        if(Competing(MATCH.c) == 'y'):
           print("You are competing!")
        elif(Competing(MATCH.c) == 'n'):
           print("You are not competing!")


    @Rule(AS.f1 << Competing('n'))
    def not_cometing(self,f1):
        self.declare(DietPlan("casual"))

    @Rule(AS.f1<<Competing("y"),
      AS.f2<<Height(MATCH.h),
      AS.f3<<Height(MATCH.w))
    def if_competing(self,h,w):
        heightInM = int(h)/100
        bmi = int(w)/pow(heightInM,2)
        if(bmi < 18.5):
            self.declare(BMI("underweight"))
            print("You are underweight")
        elif(bmi >= 18.5 and bmi < 25.0):
            self.declare(BMI("normal"))
            print("You are normal")
        elif(bmi >= 25.0):
            self.declare(BMI("overweight")) 
            print("You are fat lol")

    @Rule(AS.f1<<Intensity("low"),OR(BMI("normal"),
              BMI("overweight")
             ))
    def low_normal_overweight(self):
        self.declare(DietPlan("light"))

    @Rule(AS.f1<<Intensity("low"),BMI("underweight")
      )
    def low_underweight(self):
         self.declare(DietPlan("moderate"))
    @Rule(AS.f1 << Intensity("moderate"))
    def moderate_normal(self):
        self.declare(DietPlan("moderate"))

    @Rule(AS.f1 << Intensity("high"),
          BMI("overweight"))
    def high_overweight(self):
        self.declare(DietPlan("moderate"))


    @Rule(AS.f1 << Intensity("high"),
          OR(
              BMI("normal"),
              BMI("underweight")
             )
          )
    def high_normal_underweight(self):
        self.declare(DietPlan("intensive"))

    ''' Rules Section - Display Diet Plans '''
    
    
    @Rule(DietPlan(MATCH.f),
          salience=2)
    def display_assistance(self, f):
        print("For a " + f + " diet plan, these are the options")
    
    @Rule(AS.f1 << DietPlan(MATCH.f),
          Option(foodSet = MATCH.f,
                 option = MATCH.o,
                 calories = MATCH.c),salience=1)
    def display_moderate_plans(self, o, c):
        print(o + "(" + c + "kcal)")
    
    @Rule(AS.f1 << DietPlan("casual"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "casual",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_casual(self, cc):
        if (cc < 1500.0):
            self.declare(setOptions("A"))
        elif (cc >= 1500.0 and cc < 2500.0):
            self.declare(setOptions("B"))
        elif (cc >=2500.0):
            self.declare(setOptions("C"))
            
    @Rule(AS.f1 << DietPlan("light"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "light",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_light(self, cc):
        if (cc < 1500.0):
            self.declare(setOptions("D"))
        elif (cc >= 1500.0 and cc < 2500.0):
            self.declare(setOptions("E"))
        elif (cc >= 2500.0):
            self.declare(setOptions("F"))
    
    @Rule(AS.f1 << DietPlan("moderate"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "moderate",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_moderate(self, cc):
        if (cc < 2500.0):
            self.declare(setOptions("G"))
        elif (cc >= 2500.0 and cc < 3500.0):
            self.declare(setOptions("H"))
        elif (cc >= 3500.0):
            self.declare(setOptions("I"))
            
    @Rule(AS.f1 << DietPlan("intensive"),
          AS.f2 << Calories(value=MATCH.cc),
          Option(foodSet = "intensive",
                 option = MATCH.o,
                 calories = MATCH.c))
    def set_calorie_range_intensive(self, cc):
        if (cc < 3500.0):
            self.declare(setOptions("G"))
        elif (cc >= 3500.0 and cc < 4500.0):
            self.declare(setOptions("H"))
        elif (cc >= 4500.0):
            self.declare(setOptions("I"))

    def __init__(self):
        super().__init__()
        self.result = None
    
    @Rule(setOptions(MATCH.o),
          Calories(value=MATCH.cc),
          Set(setCategory = MATCH.o,
              setFoods = MATCH.f,
              setCalories = MATCH.c))
    def capture_result(self, o, cc, f, c):
        print("Considering you need " , cc , "kcal, you are recommended eat these in set " + o +": ")
        for z in f:
            print(z + "\t", end="")
        print()
        print("All foods in this set contains calories in the " + c + " range.")
        self.result = {
            "result": {
                "Considering you need": cc,
                "kcal, you are recommended eat these in set ":o,
                "recommendation":f,
                "All foods in this set contains calories in the range":c,
        }
    }

    @Rule(setOptions(MATCH.o),
          Calories(value=MATCH.cc),
          Set(setCategory = MATCH.o,
              setFoods = MATCH.f,
              setCalories = MATCH.c))
    def display_foods(self, o, cc, f, c):
        print("Considering you need " , cc , "kcal, you are recommended eat these in set " + o +": ")
        for z in f:
            print(z + "\t", end="")
        print()
        print("All foods in this set contains calories in the " + c + " range.")

In [4]:
class AdvancedSystemDiet(KnowledgeEngine):

    @DefFacts()
    def system_rules(self):
        
        yield Option(foodSet="casual",option="C",calories=">=2500") 
        yield Option(foodSet="casual",option="B",calories=">=1500 & <2500")
        yield Option(foodSet="casual",option="A",calories="<1500") 
            
        yield Option(foodSet="light",option="F",calories=">=1700")
        yield Option(foodSet="light",option="E",calories=">=1300 & <1700")
        yield Option(foodSet="light",option="D",calories="<1300")
            
        yield Option(foodSet="moderate",option="I",calories=">=2100")
        yield Option(foodSet="moderate",option="H",calories=">=1700 & <2100")
        yield Option(foodSet="moderate",option="G",calories="<1700")
            
        yield Option(foodSet="intensive",option="L",calories=">=2500")
        yield Option(foodSet="intensive",option="K",calories=">=2100 & <2500")
        yield Option(foodSet="intensive",option="J",calories="<2100")
        
        
        # yield Set(
        #       setCategory="A",
        #       setFoods=["Nasi Lemak","Nasi Ayam Bakar","Mee Hailam"],
        #       setCalories="<1500",
        #       setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set A can supply right amount of calorie, \nunder 1500kcal, which suits your need. \n\nTo maintain the energy reservation."
        #       )
        # yield Set(
        #     setCategory="B",
        #     setFoods=["Roti Telur","Nasi Arab Mandy","Mee Goreng Mamak"],
        #     setCalories=">=1500 & <2500",
        #     setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set B can supply right amount of calorie, \nbetween 1500kcal to 2500kcal, which suits your \nneed. \nTo maintain the energy reservation."
        #     )
        # yield Set(
        #     setCategory="C",
        #     setFoods=["Nasi Kerabu", "Roti Canai","Nasi Kandar","Chicken Chop"],
        #     setCalories=">=2500",
        #     setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set C can supply right amount of calorie, \nabove 2500kcal, which suits your need. \n\nTo maintain the energy reservation."
        #     )
        # yield Set(
        #     setCategory="D",
        #     setFoods=["Sandwich","Nasi Ayam Bakar","Bihun Tom Yam"],
        #     setCalories="<1300",
        #     setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set D can supply right amount of calorie,\nunder 1300kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
        #     )
        # yield Set(
        #     setCategory="E",
        #     setFoods=["Roti Telur","Nasi Briyani","Spaghetti", "Telur Rebus"],
        #     setCalories=">=1300 & <1700",
        #     setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set E can supply right amount of calorie,\nbetween 1300kcal to 1700kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
        #     )
        # yield Set(
        #     setCategory="F",
        #     setFoods=["Kuey Teow Goreng", "Roti Telur","Nasi Kandar","Chicken Chop"],
        #     setCalories=">=1700",
        #     setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set F can supply right amount of calorie,\nabove 1700kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
        #     )
        # yield Set(
        #     setCategory="G",
        #     setFoods=["Nasi Lemak","Nasi Ayam Bakar","Pizza"],
        #     setCalories="<1700",
        #     setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set G can supply right amount of calorie, \nunder 1700kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
        #     )
        # yield Set(
        #     setCategory="H",
        #     setFoods=["Roti Canai","Nasi Goreng Ayam", "Laksa","Nasi Goreng Telur"],
        #     setCalories=">=1700 & <2100",
        #     setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set H can supply right amount of calorie, \nbetween 1700kcal to 2100kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
        #     )
        # yield Set(
        #     setCategory="I",
        #     setFoods=["Kuey Teow Goreng","Roti Telur", "Nasi Arab Mandy", "Bihun Sup", "Spaghetti", "Chicken Chop"], 
        #     setCalories=">=2100",
        #     setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set I can supply right amount of calorie, \nabove 2100kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
        #     )
        # yield Set(
        #     setCategory="J",
        #     setFoods=["Kuey Teow Goreng","Nasi Briyani", "Mee Goreng Mamak"], 
        #     setCalories="<2100",
        #     setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set J can supply right amount of calorie, \nunder 2100kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
        #     )
        # yield Set(
        #     setCategory="K",
        #     setFoods=["Nasi Lemak", "Roti Telur","Nasi Ayam Bakar", "Pizza"],
        #     setCalories=">=2100 & <2500",
        #     setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set K can supply right amount of calorie, \nbetween 2100kcal to 2500kcal, which suits your \nneed. \nTo maintain the energy reservation for your compeitition."
        #     )
        # yield Set(
        #     setCategory="L",
        #     setFoods=["Roti Canai", "Nasi Kerabu", "Nasi Goreng Ayam", "Laksa", "Nasi Goreng Telur"],
        #     setCalories=">=2500",
        #     setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set L can supply right amount of calorie, \nabove 2500kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
        #     )
        
        yield Set(
              setCategory="A",
              setFoods=["Fatty Rice","Grilled Chicken Rice","Mee Hailam"],
              setCalories="<1500",
              setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set A can supply right amount of calorie, \nunder 1500kcal, which suits your need. \n\nTo maintain the energy reservation."
              )
        yield Set(
            setCategory="B",
            setFoods=["Egg Bread","Mandy Arabic Rice","Mee Goreng Mamak"],
            setCalories=">=1500 & <2500",
            setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set B can supply right amount of calorie, \nbetween 1500kcal to 2500kcal, which suits your \nneed. \nTo maintain the energy reservation."
            )
        yield Set(
            setCategory="C",
            setFoods=["Nasi Kerabu", "Roti Canai","Nasi Kandar","Chicken Chop"],
            setCalories=">=2500",
            setExplanation="We recommend Casual Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set C can supply right amount of calorie, \nabove 2500kcal, which suits your need. \n\nTo maintain the energy reservation."
            )
        yield Set(
            setCategory="D",
            setFoods=["Sandwich","Baked Chicken Rice","Tom Yam Vermicelli"],
            setCalories="<1300",
            setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set D can supply right amount of calorie,\nunder 1300kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
            )
        yield Set(
            setCategory="E",
            setFoods=["Egg Bread","Briyani Rice","Spaghetti", "Poached Eggs"],
            setCalories=">=1300 & <1700",
            setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set E can supply right amount of calorie,\nbetween 1300kcal to 1700kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
            )
        yield Set(
            setCategory="F",
            setFoods=["Fried Teow Kuey", "Egg Bread", "Nasi Kandar", "Chicken Chop"],
            setCalories=">=1700",
            setExplanation="We recommend Low Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set F can supply right amount of calorie,\nabove 1700kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
            )
        yield Set(
            setCategory="G",
            setFoods=["Nasi Lemak", "Baked Chicken Rice","Pizza"],
            setCalories="<1700",
            setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set G can supply right amount of calorie, \nunder 1700kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
            )
        yield Set(
            setCategory="H",
            setFoods=["Roti Canai","Chicken Fried Rice", "Laksa","Egg Fried Rice"],
            setCalories=">=1700 & <2100",
            setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set H can supply right amount of calorie, \nbetween 1700kcal to 2100kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
            )
        yield Set(
            setCategory="I",
            setFoods=["Greng Teow Kuey", "Egg Bread", "Mandy Arabic Rice", "Soup Noodles", "Spaghetti", "Chicken Chop"], 
            setCalories=">=2100",
            setExplanation="We recommend Moderate Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal and BMI. \nThe food in Set I can supply right amount of calorie, \nabove 2100kcal, which suits your need. \n\nThe recommendation can help you to maintain the energy reservation."
            )
        yield Set(
            setCategory="J",
            setFoods=["Greng Teow Kuey", "Rice Briyani", "Mamak Fried Noodles"], 
            setCalories="<2100",
            setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set J can supply right amount of calorie, \nunder 2100kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
            )
        yield Set(
            setCategory="K",
            setFoods=["Fat Rice", "Egg Bread", "Baked Chicken Rice", "Pizza"],
            setCalories=">=2100 & <2500",
            setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set K can supply right amount of calorie, \nbetween 2100kcal to 2500kcal, which suits your \nneed. \nTo maintain the energy reservation for your compeitition."
            )
        yield Set(
            setCategory="L",
            setFoods=["Roti Canai", "Nasi Kerabu", "Chicken Fried Rice", "Laksa", "Egg Fried Rice"],
            setCalories=">=2500",
            setExplanation="We recommend High Intensive Meal Planning for your diet stratergy. \n\nWith the given calorie goal. \nThe food in Set L can supply right amount of calorie, \nabove 2500kcal, which suits your need. \n\nTo maintain the energy reservation for your compeitition."
            )


    @Rule()
    def startup(self):
        print("Welcome to Diet Recommendation system")
        # questionAge = "Please enter your age: "
        # resAge = input(questionAge).lower()
        # questionHeight = "Please enter your height (in cm): "
        # resHeight = input(questionHeight).lower()
        # questionWeight = "Please enter your weight (in kg): "
        # resWeight = input(questionWeight).lower()
        # questionGender = "Please enter your gender: "
        # resGender = input(questionGender).lower()
        # self.declare(Age(resAge))
        # self.declare(Height(resHeight))
        # self.declare(Weight(resWeight))
        # self.declare(Gender(resGender))
        
    @Rule(AS.f1 << Gender(MATCH.g),
          AS.f2 << Age(MATCH.a),
          AS.f3 << Height(MATCH.h),
          AS.f4 << Weight(MATCH.w))
    def gender(self, g, a, h, w):
        if(g == "male"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))+5)
            self.declare(Calories(value = caloriesFloat))
        elif(g =="female"):
            caloriesFloat = float((10*float(w))+(6.25*float(h))-(5*float(a))-161)
            self.declare(Calories(value = caloriesFloat))
        else:
            print("Invalid gender!")
        self.declare(Ask("competition-participation")) 
            
    @Rule(AS.f1 << Ask("competition-participation"))
    def if_competing(self, f1):
        
        # resCompeting = input("Are you participating in a competition? [y/n] : ").lower()
        # self.declare(Competing(resCompeting))
        self.retract(f1)
        
            
    @Rule(AS.f1 << Competing('n'))
    def not_competing(self, f1):
        
        self.declare(DietPlan("casual"))
        
    @Rule(AS.f1 << Competing('y'),
          AS.f2 << Height(MATCH.h),
          AS.f3 << Weight(MATCH.w))
    def is_competing(self, h, w):
        print("How intense are your training?")
        # resIntensity = input("Choose one of the options [low/moderate/high]: ").lower()
        heightInM = int(h)/100
        bmi = round(int(w)/pow(heightInM,2), 2)
        self.declare(BMIValue(bmi))
        # self.declare(Intensity(resIntensity))
        if(bmi < 18.5):
            self.declare(BMI("underweight"))
        elif(bmi >= 18.5 and bmi < 28.0):
            self.declare(BMI("normal"))
        elif(bmi >= 28.0):
            self.declare(BMI("overweight"))
        
        
     
    # low intensity
    @Rule(AS.f1 << Intensity("low"),
          OR(
              BMI("normal"),
              BMI("overweight")
             )
          )
    def low_normal_overweight(self):
        self.declare(DietPlan("light"))
        
    @Rule(AS.f1 << Intensity("low"),
          BMI("underweight"))
    def low_underweight(self):
        self.declare(DietPlan("moderate"))
        
    
    # moderate intensity
    
    @Rule(AS.f1 << Intensity("moderate"))
    def moderate_normal(self):
        self.declare(DietPlan("moderate"))
        
    # high intensity
        
    @Rule(AS.f1 << Intensity("high"),
          BMI("overweight"))
    def high_overweight(self):
        self.declare(DietPlan("moderate"))
        
    @Rule(AS.f1 << Intensity("high"),
          OR(
              BMI("normal"),
              BMI("underweight")
             )
          )
    def high_normal_underweight(self):
        self.declare(DietPlan("intensive"))
        
    @Rule(AS.f1 << DietPlan("casual"),
        AS.f2 << Calories(value=MATCH.cc),
        Option(foodSet = "casual",
                option = MATCH.o,
                calories = MATCH.c))
    def set_calorie_range_casual(self, cc):
        if (cc < 1500.0):
            self.declare(setOptions("A"))
        elif (cc >= 1500.0 and cc < 2500.0):
            self.declare(setOptions("B"))
        elif (cc >=2500.0):
            self.declare(setOptions("C"))
                
    @Rule(AS.f1 << DietPlan("light"),
        AS.f2 << Calories(value=MATCH.cc),
        Option(foodSet = "light",
                option = MATCH.o,
                calories = MATCH.c))
    def set_calorie_range_light(self, cc):
        if (cc < 1300.0):
            self.declare(setOptions("D"))
        elif (cc >= 1300.0 and cc < 1700.0):
            self.declare(setOptions("E"))
        elif (cc >= 1700.0):
            self.declare(setOptions("F"))
        
    @Rule(AS.f1 << DietPlan("moderate"),
        AS.f2 << Calories(value=MATCH.cc),
        Option(foodSet = "moderate",
                option = MATCH.o,
                calories = MATCH.c))
    def set_calorie_range_moderate(self, cc):
        if (cc < 1700.0):
            self.declare(setOptions("G"))
        elif (cc >= 1700.0 and cc < 2100.0):
            self.declare(setOptions("H"))
        elif (cc >= 2100.0):
            self.declare(setOptions("I"))
                
    @Rule(AS.f1 << DietPlan("intensive"),
        AS.f2 << Calories(value=MATCH.cc),
        Option(foodSet = "intensive",
                option = MATCH.o,
                calories = MATCH.c))
    def set_calorie_range_intensive(self, cc):
        if (cc < 2100.0):
            self.declare(setOptions("J"))
        elif (cc >= 2100.0 and cc < 2500.0):
            self.declare(setOptions("K"))
        elif (cc >= 2500.0):
            self.declare(setOptions("L"))

    
    def __init__(self):
        super().__init__()
        self.result = None



    
    @Rule(
        Gender(MATCH.g),
        Age(MATCH.a),
        Calories(value=MATCH.cc),
        BMIValue(MATCH.bVal),
        BMI(MATCH.b),
        setOptions(MATCH.o),
        Set(
            setCategory=MATCH.o,
            setFoods=MATCH.f,
            setCalories=MATCH.c,
            setExplanation=MATCH.ef
        )
    )
    def capture_result(self, g, a, cc, bVal, b, o, f, c, ef):
        self.result = {
            "result": {
                "Since your gender is": g,
                "and your age is": a,
                "We have calculated your calories based using Mifflin St Jeor Equation And your calorie_goal": cc,
                "bmi": {
                    "value": bVal,
                    "category": b
                },
                "Considering you need ": cc,
                "recommended_foods": f,
                "You are recommended eat these in set ": o,
                "All foods in this set contains calories in the": c,
                "explanation": ef
            }
        }
    

    @Rule(Gender(MATCH.g),
          Age(MATCH.a),
          Calories(value=MATCH.cc),
          BMIValue(MATCH.bVal),
          BMI(MATCH.b),
          setOptions(MATCH.o),
          Set(setCategory = MATCH.o,
              setFoods = MATCH.f,
              setCalories = MATCH.c,
              setExplanation = MATCH.ef))
    def display_foods(self, g, a, cc, bVal, b, o,  f, c, ef):
        print("\n** Result **")
        print("Since your gender is " , g , " and your age is ", a)
        print("We have calculated your calories based using Mifflin St Jeor Equation")
        print("And your calorie goal is " , cc)
        print("\n** Recommendation **")
        print("Based on your BMI ", bVal , " it is " , b)
        print("Considering you need ", cc ,"kcal")
        print("You are recommended eat these in set " + o +": ")
        for z in f:
            print(z + "\t", end="")
        print()
        print("All foods in this set contains calories in the " + c + " range.")
        print()
        print("** Explanation **")
        print(ef)
        

In [5]:
from fastapi import FastAPI
import asyncio 
from uvicorn import Config , Server
from pydantic import BaseModel
app = FastAPI()

class UserInput(BaseModel):
    age : str
    height : str
    weight : str
    gender : str
    competing : str
    intensity : str

@app.get('/')
async def index():
    return {"Message":"Hello World"}

config  = Config(app)
server  =  Server(config=config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

@app.post('/diet')
async def age(user_input: UserInput):
    age = user_input.age
    height = user_input.height
    weight = user_input.weight
    gender = user_input.gender
    competing = user_input.competing
    intensity = user_input.intensity
    engine = SystemDiet()
    engine.reset()
    engine.declare(Age(age), Height(height), Weight(weight), Gender(gender), Competing(competing), Intensity(intensity))
    print(engine.run())
    engine.run()
    result = engine.result
    print(result)
    return {"result":result}

@app.post('/advanced-diet')
async def age(user_input: UserInput):
    age = user_input.age
    height = user_input.height
    weight = user_input.weight
    gender = user_input.gender
    competing = user_input.competing
    intensity = user_input.intensity
    engine = AdvancedSystemDiet()
    engine.reset()
    engine.declare(Age(age), Height(height), Weight(weight), Gender(gender), Competing(competing), Intensity(intensity))
    engine.run()
    result = engine.result
    print(result)
    return {"result":result}
 

In [7]:
# import numpy as np
# import skfuzzy as fuzz
# from skfuzzy import control as ctrl

# class SystemDietUnc:
#     def init(self):
#         self.result = None

#     def run(self):

#         age = ctrl.Antecedent(np.arange(0, 101, 1), 'age')
#         height = ctrl.Antecedent(np.arange(0, 301, 1), 'height')
#         weight = ctrl.Antecedent(np.arange(0, 201, 1), 'weight')
#         diet_plan = ctrl.Consequent(np.arange(0, 11, 1), 'diet_plan')


#         age['young'] = fuzz.trimf(age.universe, [0, 0, 50])
#         age['old'] = fuzz.trimf(age.universe, [30, 100, 100])

#         height['short'] = fuzz.trimf(height.universe, [0, 0, 150])
#         height['tall'] = fuzz.trimf(height.universe, [120, 300, 300])

#         weight['light'] = fuzz.trimf(weight.universe, [0, 0, 100])
#         weight['heavy'] = fuzz.trimf(weight.universe, [60, 200, 200])

#         diet_plan['casual'] = fuzz.trimf(diet_plan.universe, [0, 0, 5])
#         diet_plan['intensive'] = fuzz.trimf(diet_plan.universe, [3, 10, 10])


#         rule1 = ctrl.Rule(age['young'] & height['tall'] & weight['light'], diet_plan['casual'])
#         rule2 = ctrl.Rule(age['old'] | weight['heavy'], diet_plan['intensive'])

#         system_diet = ctrl.ControlSystem([rule1, rule2])


#         simulation = ctrl.ControlSystemSimulation(system_diet)


#         simulation.input['age'] = 30
#         simulation.input['height'] = 180
#         simulation.input['weight'] = 80
#         simulation.compute()
#         diet_plan_result = simulation.output['diet_plan']
#         print('Recommended diet plan:', diet_plan_result)

# system = SystemDietUnc()

# system.run()

How intense are your training?

** Result **
Since your gender is  male  and your age is  21
We have calculated your calories based using Mifflin St Jeor Equation
And your calorie goal is  1545.0

** Recommendation **
Based on your BMI  19.27  it is  normal
Considering you need  1545.0 kcal
You are recommended eat these in set E: 
Egg Bread	Briyani Rice	Spaghetti	Poached Eggs	
All foods in this set contains calories in the >=1300 & <1700 range.

** Explanation **
We recommend Low Intensive Meal Planning for your diet stratergy. 

With the given calorie goal. 
The food in Set E can supply right amount of calorie,
between 1300kcal to 1700kcal, which suits your need. 

To maintain the energy reservation for your compeitition.
Welcome to Diet Recommendation system
{'result': {'Since your gender is': 'male', 'and your age is': '21', 'We have calculated your calories based using Mifflin St Jeor Equation And your calorie_goal': 1545.0, 'bmi': {'value': 19.27, 'category': 'normal'}, 'Consideri